In [1]:
import pickle

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch

import numpy as np
import pandas as pd

import torch.nn as nn
import torch.optim as optim

from random import shuffle


In [2]:
torch.cuda.set_device(0)
device = torch.device("cuda")
torch.cuda.is_available()


True

In [3]:
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)
    torch.cuda.manual_seed_all(1337)

torch.manual_seed(1337)

np.random.seed(1337)

torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False


In [4]:
with open("train.pkl", "rb") as f:
    data_train = pickle.load(f)

with open("test_no_target.pkl", "rb") as f:
    data_test = pickle.load(f)


In [5]:
shuffle(data_train)


In [6]:
X_train = []
y_train = []
X_test = []

for data, label in data_train:
    X_train.append(torch.Tensor(data))
    y_train.append(torch.Tensor([label]))

for data in data_test:
    X_test.append(torch.Tensor(data))


In [7]:
train_max_len = max([len(sequence) for sequence in X_train])
test_max_len = max([len(sequence) for sequence in X_test])
max_len = max([train_max_len, test_max_len])


In [8]:
def pad_collate(batch, pad_value=0):
    """
    batch: list[tuple[torch.Tensor]]
    """
    xx, yy = [list(element) for element in zip(*batch)]
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx[0] = nn.ConstantPad1d((0, max_len - xx[0].shape[0]), 0)(xx[0])
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens


In [9]:
def pad_collate_test(batch, pad_value=0):
    """
    batch: list[tuple[torch.Tensor]]
    """
    xx = list(batch)
    x_len = xx[0].shape[0]

    xx[0] = nn.ConstantPad1d((0, max_len - x_len), 0)(xx[0])
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)

    return xx_pad, x_len


In [10]:
class VariableLenDataset(Dataset):
    def __init__(self, in_data, target):
        self.data = [(x, y) for x, y in zip(in_data, target)]      

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target


In [11]:
train_indices = int(len(data_train) * 0.7)

train_set = VariableLenDataset(X_train[:train_indices], y_train[:train_indices])
valid_set = VariableLenDataset(X_train[train_indices:], y_train[train_indices:])

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=pad_collate)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=False, drop_last=False, collate_fn=pad_collate)
test_loader = DataLoader(X_test, batch_size=16, shuffle=False, drop_last=False, collate_fn=pad_collate_test)


In [12]:
len(train_loader), len(valid_loader)

(129, 56)

In [13]:
iter_ = iter(train_loader)
a = next(iter_)
b = next(iter_)


In [14]:
max_len

8966

In [15]:
a[0][0].shape


torch.Size([8966])

In [16]:
class BestNetEver2(nn.Module):
    def __init__(self, input_size, hidden_size, max_len, num_layers_LSTM, out_size, dropout):
        super().__init__()
        self.num_layers_LSTM = num_layers_LSTM
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers_LSTM, dropout=dropout)                  
        self.fc_1 = nn.Linear(in_features=max_len * hidden_size, out_features=400)
        self.fc_2 = nn.Linear(in_features=400, out_features=out_size)
        self.act = nn.LeakyReLU()

    def init_hidden(self, batch_size):
        hidden_state = torch.zeros(self.num_layers_LSTM, batch_size, self.hidden_size)
        cell_state = torch.zeros(self.num_layers_LSTM, batch_size, self.hidden_size)
        return hidden_state, cell_state

    def forward(self, x, hidden):
        x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs, 0, 1)
        out = torch.flatten(all_outputs, 1)
        x = self.act(self.fc_1(out))
        x = self.fc_2(x)

        return x, hidden


model = BestNetEver2(input_size=1, hidden_size=17, max_len=max_len, num_layers_LSTM=2, out_size=5, dropout=0.6).to(device)


In [17]:
classes = pd.Series([int(y) for y in y_train]).sort_values()
class_weights = classes.value_counts().sort_index()/classes.shape[0]

optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=0.001)
loss_fun = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float).to(device))  # unballanced set
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.99)


In [18]:
model.train()
for epoch in range(15):
    epoch_lossess = []
    for x, targets, _, _ in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.type(torch.LongTensor)
        targets = targets.to(device)
        hidden_state, cell_state = model.init_hidden(x.size(0))
        hidden_state, cell_state = hidden_state.to(device), cell_state.to(device)
        preds, _ = model(x, (hidden_state, cell_state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()

        loss = loss_fun(preds, targets.squeeze(1))
        loss.backward()
        
        epoch_lossess.append(loss.item())
        optimizer.step()

    loss_mean = np.array(epoch_lossess).mean()
    torch.save(model.state_dict(), './model.pt')
    print(loss_mean)


1.0074012868626172
0.6129858700572982
0.5798968233803447
0.563606707393661
0.5558252698460291
0.5141063116548598
0.5083884172892386
0.493249299798825
0.4820827638456064
0.467766653544219
0.44127727998781574
0.43961895944536195
0.4347910625759021
0.43638852273308953
0.43161939096081164


In [19]:
model.eval()
with torch.no_grad():
    correct, all = 0, 0
    for X, labels, _, _ in valid_loader:
        X = X.to(device).unsqueeze(2)

        hidden_state, state = model.init_hidden(X.size(0))
        hidden_state, state = hidden_state.to(device), state.to(device)

        preds, _ = model(X, (hidden_state,state))
        preds = torch.argmax(preds, 1).cpu()

        correct += (preds == labels).sum().item()
        all += X.size(0)*16

f'Valid accuracy: {round(correct/all, 2)}'


'Valid accuracy: 0.52'

In [20]:
model.eval()
with torch.no_grad():
    results = list()
    for X, _ in test_loader:
        X = X.to(device).unsqueeze(2)

        hidden_state, state = model.init_hidden(X.size(0))
        hidden_state, state = hidden_state.to(device), state.to(device)

        preds, _ = model(X, (hidden_state,state))
        preds = torch.argmax(preds, 1).cpu()

        results += [int(i) for i in preds]


In [21]:
print(results)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 